In [ ]:
from pathlib import Path
import requests
import pandas as pd
import numpy as np
from io import StringIO
from simple_salesforce import Salesforce
import shutil
import datetime
from datetime import datetime
import json,re

In [ ]:
with open("login.json", "r") as j:
    misDatos = json.load(j)

In [ ]:
#para smart query
""" sf = simple_salesforce.Salesforce(
    username=misDatos["username"],
    password=misDatos["password"],
    security_token=misDatos["security_token"],
)
 """
#para lo demas
sf = Salesforce(
    username=misDatos["username"],
    password=misDatos["password"],
    security_token=misDatos["security_token"],
)
# paramiko.util.log_to_file("paramiko.log")
regex = re.compile(r"([A-Za-z0-9]+[.-_])*[A-Za-z0-9]+@[A-Za-z0-9-]+(\.[A-Z|a-z]{2,})+")

In [ ]:
Fecha_Inicio = "2023-08-01"

In [ ]:
""" sf = Salesforce(
    username="jmartinez@careassistance.com",
    password="Jcmm01041983*",
    security_token="dxbNevVOmgmoQS4Zllh3DpNDj",
) """

In [ ]:
# CArga de PAcientes
results = sf.query_all(
    """
    select Id, IdentificationId__pc from Account where ispersonaccount = True
   """
)
Pacientes_SF = pd.DataFrame(results["records"])

Pacientes_SF = Pacientes_SF[
    [
        "Id",
        "IdentificationId__pc",
    ]
]
Pacientes_SF = Pacientes_SF.rename(
    columns={
        "Id": "Account",
        "IdentificationId__pc": "Rut",
    }
)
Pacientes_SF.shape

In [ ]:
nuevos = "F:/Procesos Automaticos/AccuHealth/1.xlsx"
Archivo = pd.read_excel(nuevos,sheet_name='Accuhealth')

# OJO se copia rut1 a rut le saca el guoion
Archivo.head()

In [ ]:
Archivo["RUT"] = Archivo["RUT"].astype(str)
Archivo["RUT"] = Archivo["RUT"].str.replace("-", "")
Archivo["RUT"] = Archivo["RUT"].str.replace(".", "")
Archivo["RUT"] = Archivo["RUT"].str.upper()


In [ ]:
Archivo.dtypes

In [ ]:
def siPrimerCero(y):
    if y[0] == "0":
        y = y[1:]
    return y

In [ ]:
Archivo["RUT2"] = Archivo["RUT"].apply(lambda y: siPrimerCero(y))
Archivo.head()

In [ ]:
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].astype(str)
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].str.replace("-", "")
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].str.replace(".", "")
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].str.upper()

In [ ]:
Archivo = pd.merge(
    left=Archivo,
    right=Pacientes_SF,
    how="left",
    left_on="RUT2",
    right_on="Rut",
)
Archivo.head()

In [ ]:
Archivo["Presión Arterial"] = np.where(Archivo["DBP"].str.len() <=4,Archivo["SBP"] + " / " + Archivo["DBP"],"0")

Archivo.head()

In [ ]:
Archivo["Dia"] = pd.to_datetime(Archivo["date"]).dt.day

Archivo["Mes"] = pd.to_datetime(Archivo["date"]).dt.month

Archivo["Mes Nombre"] = pd.to_datetime(Archivo["date"]).dt.month_name()

Archivo["Año"] = pd.to_datetime(Archivo["date"]).dt.year

Archivo["Fecha_chilena"] = pd.to_datetime(
    Archivo["date"], dayfirst=True, yearfirst=False, exact=True
)
# x = date(Citas_de_servicio['Dia'], Citas_de_servicio['Mes'], Citas_de_servicio['Año'] )
# Archivo['LocalDateTime'] = pd.to_datetime(Archivo['LocalDateTime'])
# 

Archivo["Fecha_chilena"] = Archivo["Fecha_chilena"].apply(
    lambda a: datetime.strftime(a, "%Y-%m-%d %H:%M:%S")
)

Archivo["Fecha_chilena"] = pd.to_datetime(Archivo["Fecha_chilena"])

Archivo["Fecha"] = Archivo["Fecha_chilena"].dt.strftime("%d-%m-%Y")
Archivo.to_excel("F:/Procesos Automaticos/AccuHealth/2.xlsx", index=False)


Archivo = Archivo[(Archivo["severity"] != "No realizado")]

Discriminacion fecha datos relevantes para carga

In [ ]:
Archivo1 = Archivo[(Archivo["Fecha_chilena"] >= Fecha_Inicio)]
Archivo1.sort_values(by="date")

In [ ]:
Archivo1 = Archivo1[
    [
        "patient_lot_id",
        "Rut",
        "Fecha",
        "Fecha_chilena",
        "severity",
        "BG",
        "bienestar",
        "Medicacion",
        "Account",
        "Presión Arterial",
    ]
]
Archivo1.shape

In [ ]:
results = sf.query_all(
    """
    select Id,idAccuHealth__c from Ficha_Cl_nica_Seg_Paciente_Cr_nico__c 
   """
)
fichaAccu = pd.DataFrame(results["records"])

fichaAccu = fichaAccu[
    [
        "Id",
        "idAccuHealth__c",
    ]
]
fichaAccu = fichaAccu.rename(
    columns={
        "Id": "Id Account Paciente",
        "idAccuHealth__c": "Codigo",
    }
)
fichaAccu["Codigo"] = fichaAccu["Codigo"].astype(str)
fichaAccu["Codigo"] = fichaAccu["Codigo"].str.replace("-", "")
fichaAccu["Codigo"] = fichaAccu["Codigo"].str.replace(".", "")
fichaAccu["Codigo"] = fichaAccu["Codigo"].str.upper()

fichaAccu.shape

In [ ]:
fichaAccu.head()

In [ ]:
fichaAccu["Codigo"] = fichaAccu["Codigo"].astype(str)


In [ ]:

Archivo1["patient_lot_id"] = Archivo1["patient_lot_id"].astype(str)
Archivo1["patient_lot_id"] = Archivo1["patient_lot_id"].str.replace("-", "")
Archivo1["patient_lot_id"] = Archivo1["patient_lot_id"].str.replace(".", "")
Archivo1["patient_lot_id"] = Archivo1["patient_lot_id"].str.upper()

Archivo1["Estaono"] = Archivo1["patient_lot_id"].isin(fichaAccu["Codigo"])
Archivo1["Canal__c"] = "Chatter"
Archivo1["RecordTypeId"] = "0128c000002CYuWAAW"

Archivo1.head()

In [ ]:
Archivo1 = Archivo1.rename(
    columns={
        "patient_lot_id": "idAccuHealth__c",
        "Rut": "Rut__c",
        "Fecha_chilena": "Fecha__c",
        "BG": "Glicemia__c",
        "bienestar": "Bienestar__c",
        "Medicacion": "Medicaci_n__c",
        "Account": "Account__c",
        "Presión Arterial": "Presi_n_Arterial__c",
        "Estaono": "Estaono",
    }
)
Archivo1 = Archivo1[Archivo1["Estaono"] == False]

Archivo1 = Archivo1[
    [
        "idAccuHealth__c",
        "Fecha__c",
        "Glicemia__c",
        "Bienestar__c",
        "Medicaci_n__c",
        "Account__c",
        "Presi_n_Arterial__c",
        "Canal__c",
        "RecordTypeId",
        
    ]
]


Archivo1.to_excel("F:/Procesos Automaticos/AccuHealth/3.xlsx", index=False)